In [ ]:
! pip install librosa numpy pandas demucs soundfile pyfluidsynth
! sudo yum install libsndfile

In [4]:
import librosa
import numpy as np
import os
import pandas as pd
import io
from pathlib import Path
from shutil import rmtree
import subprocess as sp


# Source Separation using Demucs

In [5]:
from dataclasses import dataclass

@dataclass
class DeMixedAudio4():
    sample_rate: int
    bass : np.ndarray
    drums : np.ndarray
    other : np.ndarray
    vocals : np.ndarray

@dataclass
class DeMixedAudio6():
    sample_rate: int
    bass : np.ndarray
    drums : np.ndarray
    other : np.ndarray
    vocals : np.ndarray
    piano : np.ndarray
    guitar : np.ndarray

sp.run(["python3", "-m", "demucs.separate", "-o", "../resources/tmp", "-n", model, path_to_file])- Does the model work with different sample rates? YES
- Does the model output have the same sample rate as the input? - NO: ALWAYS 44100
- If so TODO: find a way to find the input native sample rate of the audio and use this to re-load the stems

- The output volumes are different since the input audio is z score normalised by demucs

In [ ]:
# file_name = os.path.basename(os.path.join(os.getcwd(),'..','resources/test_file.wav'))
# sp.run(["python3", "-m", "demucs.separate", "-o", "../resources/tmp", "-n", "htdemucs_6s", os.path.join(os.getcwd(),'..','resources/test_file.wav')])
# print("Demucs done, loading files")

In [21]:
x,sr = librosa.load(os.path.join(os.getcwd(),'..','resources/Nightmare.mp3'),sr=None)

In [22]:
import soundfile as sf
sf.write('../resources/Nightmare.wav',x, sr)

In [35]:
def get_stems(path_to_file,model='htdemucs',delete_temp_files=True):
    file_name = os.path.basename(path_to_file)
    sp.run(["python3", "-m", "demucs.separate", "-o", "../resources/tmp", "-n", model, path_to_file])
    print("Demucs done, loading files")
    stems={}
    if model in ['htdemucs','htdemucs_ft']:
        for stem in ['bass','drums','other','vocals']:
            x,sr = librosa.load(os.path.join("../resources/tmp", model,file_name[:file_name.find('.')],f"{stem}.wav"), sr=None)
            stems[stem] = x
        if delete_temp_files:
            print("Deleting temp files")
            rmtree(f"../resources/tmp/{model}/{file_name[:file_name.find('.')]}")
        return DeMixedAudio4(sr, bass=stems['bass'], drums=stems['drums'], other=stems['other'], vocals=stems['vocals'])#stems
    elif model == 'htdemucs_6s':
        for stem in ['bass','drums','other','vocals','piano', 'guitar']:
            x,sr = librosa.load(os.path.join("../resources/tmp", model,file_name[:file_name.find('.')],f"{stem}.wav"), sr=None)
            stems[stem] = x
        if delete_temp_files:
            print("Deleting temp files")
            rmtree(f"../resources/tmp/{model}/{file_name[:file_name.find('.')]}")
        return DeMixedAudio6(sr, bass=stems['bass'], drums=stems['drums'], other=stems['other'], vocals=stems['vocals'], piano=stems['piano'], guitar=stems['guitar'])#stems

In [37]:
song = get_stems(os.path.join(os.getcwd(),'..','resources/Nightmare.wav'), model='htdemucs_ft', delete_temp_files=False)

ffprobe: error while loading shared libraries: libopenh264.so.5: cannot open shared object file: No such file or directory


Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in /home/ec2-user/SageMaker/scalable-asset-generation/resources/tmp/htdemucs_ft
Separating track /home/ec2-user/SageMaker/scalable-asset-generation/src/../resources/Nightmare.wav


100%|██████████████████████████████████████████████████████████████████████| 380.25/380.25 [00:16<00:00, 22.82seconds/s]
100%|██████████████████████████████████████████████████████████████████████| 380.25/380.25 [00:15<00:00, 24.76seconds/s]
100%|██████████████████████████████████████████████████████████████████████| 380.25/380.25 [00:15<00:00, 24.61seconds/s]
100%|██████████████████████████████████████████████████████████████████████| 380.25/380.25 [00:15<00:00, 24.56seconds/s]


Demucs done, loading files


In [38]:
song

DeMixedAudio4(sample_rate=44100, bass=array([-9.1552734e-05, -9.1552734e-05, -9.1552734e-05, ...,
       -1.0681152e-04, -1.9836426e-04, -2.5939941e-04], dtype=float32), drums=array([-6.1035156e-05, -6.1035156e-05, -6.1035156e-05, ...,
       -4.2724609e-04, -4.4250488e-04, -8.5449219e-04], dtype=float32), other=array([-1.2207031e-04, -4.5776367e-05, -3.0517578e-05, ...,
       -1.2207031e-04, -1.2008667e-02, -6.7749023e-03], dtype=float32), vocals=array([-6.1035156e-05, -6.1035156e-05, -6.1035156e-05, ...,
       -9.0805054e-02, -2.1102905e-02,  4.2022705e-02], dtype=float32))

# Pop2Piano

In [ ]:
!apt-get install -y fluidsynth
!git clone https://github.com/sweetcocoa/pop2piano/
!cd pop2piano
!pip install pretty-midi==0.2.9 omegaconf==2.1.1 youtube-dl==2021.12.17 transformers==4.16.1 pytorch-lightning==1.8.4 essentia==2.1b6.dev609 note-seq==0.0.3 pyFluidSynth==1.3.0
!wget https://github.com/sweetcocoa/pop2piano/releases/download/dpi_2k_epoch/model-1999-val_0.67311615.ckpt

In [ ]:
import os 
import sys
sys.path.append("pop2piano")
import glob
import random

import torch
import torchaudio
import librosa
import numpy as np
import pandas as pd
import IPython.display as ipd
import soundfile as sf
# from google.colab import files

from tqdm.auto import tqdm
from omegaconf import OmegaConf
import note_seq

from utils.dsp import get_stereo
from utils.demo import download_youtube
from transformer_wrapper import TransformerWrapper
from midi_tokenizer import MidiTokenizer, extrapolate_beat_times
from preprocess.beat_quantizer import extract_rhythm, interpolate_beat_times

from mido import MidiFile

In [ ]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# config = OmegaConf.load("pop2piano/config.yaml")
# wrapper = TransformerWrapper(config)
# wrapper = wrapper.load_from_checkpoint("model-1999-val_0.67311615.ckpt", config=config).to(device)
# model = "dpipqxiy"

# composer='composer1'
# wrapper.eval()

In [ ]:
# pm, composer, mix_path, midi_path = wrapper.generate(
#     audio_path=os.path.join(os.getcwd(),'..','resources/test_file.wav'), 
#     composer=composer, 
#     model=model,
#     show_plot=False, 
#     save_midi=True, 
#     save_mix=False, 
# )
# # note_seq.plot_sequence(note_seq.midi_to_note_sequence(pm))

In [ ]:
def get_piano(path_to_file, composer='composer1'):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    config = OmegaConf.load("pop2piano/config.yaml")
    wrapper = TransformerWrapper(config)
    wrapper = wrapper.load_from_checkpoint("model-1999-val_0.67311615.ckpt", config=config).to(device)
    model = "dpipqxiy"

    wrapper.eval()
    pm, composer, mix_path, midi_path = wrapper.generate(
        audio_path=path_to_file, 
        composer=composer, 
        model=model,
        show_plot=False, 
        save_midi=True, 
        save_mix=False, 
    )
    midi_file = MidiFile(midi_path)
    os.remove(midi_path)
    
    return midi_file

In [ ]:
midi = get_piano('../resources/test_resources/test_file-48000.wav')
midi

# Output

In [ ]:
acapella = song.vocals

In [ ]:
acapella

# Stem gain testing

In [ ]:
!pip install pyloudnorm

In [ ]:
import pyloudnorm as pyln

In [ ]:
meter = pyln.Meter(song.sample_rate) # create BS.1770 meter
meter.integrated_loudness(song.vocals) # measure loudness

In [ ]:
# Original track loudness
meter.integrated_loudness(librosa.load(os.path.join(os.getcwd(),'..','resources/test_file.wav'))[0])

In [ ]:
# Reconstructed track loudness
meter.integrated_loudness(song.vocals + song.other + song.bass + song.drums)

In [ ]:
x,sr = librosa.load(os.path.join(os.getcwd(),'..','resources/test_file.wav'), sr=None)

x_48k = librosa.resample(x, orig_sr=sr, target_sr=48000)

In [ ]:
import soundfile as sf
sf.write('../resources/test_resources/test_file-48000.wav',x_48k, 48000)

In [ ]:
x_test,sr_test = librosa.load('../resources/test_resources/test_file-48000.wav', sr=None)

In [ ]:
sr_test

In [ ]:
song = get_stems(os.path.join(os.getcwd(),'..','resources/test_resources/test_file-48000.wav'))

Humanization- https://github.com/erwald/rachel

# Processing Pipeline

In [ ]:
def process_file(path_to_song:str, output_folder_path:str, )